In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="hyperdrive_experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

# HyperDrive Pipeline

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "compute_cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# get a detailed status for the current cluster
print(compute_target.get_status().serialize())

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C': uniform(0.5, 5.0),
        '--max_iter': choice(50, 100, 150, 200)
    }
)
  

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

script_dir = "./training"
if "training" not in os.listdir():
    os.mkdir(script_dir)
    
shutil.copy('train.py', script_dir)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_dir, entry_script='train.py', compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=20,
                             max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output = True)

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
hyperdrive_run

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run, best_model = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_details = best_run.get_details() 

print(best_run)
print(best_run_metrics)
print(best_run_details['runDefinition']['arguments'])

print('Best Run Id: ', best_run.id)
print('\n Accuracy: ', best_run_metrics['accuracy'])
print('\n C:',best_run_details[1])
print('\n Maximum Iteration:',best_run_details[3])

In [ ]:
print(best_run.get_file_names())

In [ ]:
os.makedirs('./outputs', exist_ok=True)
joblib.dump(best_run, 'outputs/best_model.pkl')

In [ ]:
model = best_run.register_model(model_name = 'best-hdr-model', model_path = 'outputs/model.pkl')

In [ ]:
model.download_file(name='best-hdr-model', output_file_path='./outputs')

# AutoML Run

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=dataset_url)

In [ ]:
df = ds.to_pandas_dataframe()
df.head()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x['y'] = y

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=3)

In [2]:
# Submit your automl run

exp1 = Experiment(ws, "automl_experiment")
automl_run = exp1.submit(automl_config, show_output=True)

In [ ]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
assert(automl_run.get_status() == "Completed")

In [ ]:
automl_run

In [ ]:
# Retrieve and save your best automl model.
best_automl_run, best_automl_model = automl.get_output()
best_automl_run_metrics = best_automl_run.get_metrics()
best_automl_run_details = best_automl_run.get_details()

In [ ]:
print(best_automl_run)
print(best_automl_run_metrics)
print(best_automl_run_details['runDefinition']['arguments'])

print('Best Run Id: ', best_automl_run)
print('\n Accuracy: ', best_automl_run_metrics['accuracy'])
print('\n C:',best_automl_run_details[1])
print('\n Maximum Iteration:',best_automl_run_details[3])

In [ ]:
joblib.dump(best_automl_run, 'outputs/best_automl_model.pkl')

In [ ]:
automl_model = best_automl_run.register_model(model_name = "best-automl-model", model_path = '/outputs/automl_model.pkl')

In [ ]:
automl_model.download_file(name='best-automl-model', output_file_path='./outputs')

In [ ]:
# Cluster cleanup
AmlCompute.delete(compute_target)